# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** This is a classification problem because we are simply identifying the binary: does the student need intervention? yes/no. For it to be a regression problem we would need to correlate performance to a variable scale of intervention (let's say hours of tutoring). Even a selection of no intervention, some intervention, and a lot of intervention would still be a classification problem.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = (len(student_data.columns) - 1)

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] =='yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] !='yes'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)/float(n_students) * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn import cross_validation

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
def shuffle_split_data(X, y):
    """ Shuffles and splits data into 75% training and 25% testing subsets,
        then returns the training and testing subsets. """

    # Shuffle and split the data
    X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y,stratify=y_all, test_size=0.24, random_state=1)

    # Return the training and testing data subsets
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = shuffle_split_data(X_all, y_all)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** The three supervised models I am going to use are Decision Trees, K-Nearest Neighbors, and Support Vector Machines (SVM). 

**Decision Trees:**
Real world application - The simple twenty-questions game from mattel (https://www.amazon.com/Mattel-R8559-20Q-Version-3-0/dp/B0031ES9LE) is a fine example of a decision tree. It asks a series of questions each time dividing the set of possible answers, and trys to guess what you are thinking of. 

Strengths - Decision trees are easy for people to understand. They do a pretty good job at classification assuming the underlying phenomenon are related to the data in a sensible way. 

Weaknesses - Because they are easy to understand, it is also easy to draw false conclusions. For example, I heard of a situation where the ID number being above a certain threshold increased the rate of positives in a sample for a medical test. The real reason for that was that the location was encoded into the ID number, so the relevant data (location) was obscured by irrelevant information (ID).

What makes this model a good candidate - We have some intuition about what's going on to contribute to a student needing intervention. However, our intutition may not be accurate so it as least worthwhile to investigate how well a decision tree can do, if for no other reason than to establish a baseline for our intuition.

**K-Nearest Neighbors (KNN): **

Real world application - KNN is sometimes used in stock price prediction, store purchase predictions, as well as some fraud detection and other security applications.

Strengths - KNN costs nothing to learn up front, it is a "lazy learner." In addition it does pretty well with a system where numerous factors interact determinitically but not in an extremely intuitive way. It is eseentially comparing something new to the "most similar cases," but the similarities can easily be lost on people.

Weaknesses - Computational costs can be quite high if the number of distances we are computing is large and then number of neighbors is large. This can mean it is incrementally quite slow compared to other types of learning.

What makes this model a good candidate - KNN is a very versatile and this application is neither large nor high performance enough to be penalized greatly by the incremental computational as student interventions needs will likely not need to be calculated in real time. 

**Support Vector Machines (SVM): **

Real world application - According to wikipedia, SVM's are used to classify proteins in biological applications. In addition, SVM's are like neural nets in that they are often used for numerous, numerous applications as it is a hip learning method.

Strengths - SVM's are very powerful, and do not suffer from the local optima issues that neural nets suffer from. They also allow for the representation of expert knowledge through kernal design. An explicit trade-off between complexity and error is possible and a lot more obvious than with some of the other more complex models.

Weaknesses - Like many powerful models SVMs are not transparent so in this application (student intervention classification), though the model may be accurate, it will not be as intuitive as KNN or decision trees. They are fairly intensive models, so it is necessary to have sufficient data. 

What makes this model a good candidate - SVM's are a trendy model that is intensive and powerful. So it would be a great choice just as a comparison to KNN and a decision tree. A SVM is also a great a choice because, being opaque and non-intuitive, it might be accurate in ways that a person may not have thought intitially when looking at the problem. That could provide valuable insight into what is actually going on in the system.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [12]:
# TODO: Import the three supervised learning models from sklearn
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

# TODO: Initialize the three models
clf_A = tree.DecisionTreeClassifier(random_state=42)
clf_B = KNeighborsClassifier()
clf_C = svm.SVC(random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

#TODO: Execute the 'train_predict' function for each classifier and each training set size
train_predict(clf_A, X_train_100, y_train_100, X_test, y_test)
print "___________________"
train_predict(clf_A, X_train_200, y_train_200, X_test, y_test)
print "___________________"
train_predict(clf_A, X_train_300, y_train_300, X_test, y_test)
print "___________________"
train_predict(clf_B, X_train_100, y_train_100, X_test, y_test)
print "___________________"
train_predict(clf_B, X_train_200, y_train_200, X_test, y_test)
print "___________________"
train_predict(clf_B, X_train_300, y_train_300, X_test, y_test)
print "___________________"
train_predict(clf_C, X_train_100, y_train_100, X_test, y_test)
print "___________________"
train_predict(clf_C, X_train_200, y_train_200, X_test, y_test)
print "___________________"
train_predict(clf_C, X_train_300, y_train_300, X_test, y_test)
print "___________________"

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0026 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.5714.
___________________
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0054 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0007 seconds.
F1 score for test set: 0.6613.
___________________
Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0056 seconds
Made predictions in 0.0009 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0017 seconds.
F1 score for test set: 0.7234.
___________________
Training a KNeighborsClassifier using a training set size of 100. . .
Trained model in 0.0018 seconds
Made predictions in 0.0047 seconds.
F1 score for training set: 0.8085.
Made predictions in 0.0041 seconds.
F1 sc

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |0.0026                   | 0.0007                |1.0000            |0.5714            |
| 200               |        0.0054          |     0.0007                  |1.0000               | 0.6613           |
| 300               | 0.0056                  | 0.0017                  |  1.0000         |    0.6613     |

** Classifer 2 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0018                | 0.0041                 |  0.8085           | 0.7050            |
| 200               |     0.0018             |  0.0056                 |  0.8571          | 0.7164          |
| 300               |      0.0022           | 0.0071                | 0.8546            |    0.7591     |

** Classifer 3 - ?**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0036                 | 0.0029                  |  0.8707           |      0.8000   |
| 200               |       0.0088            |  0.0045                 |   0.8491         |  0.8026         |
| 300               |        0.017         | 0.0061                   |   0.8701         |    0.8026         |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **
I think the best model out of the three tested was the Support Vector Machine Classifier(SVM/SVC). It is a bit of a heavy hammer, in that it is computationally intensive both initially and on an ongoing basis. However, it does a very good job classifying according to F1 score. We see very little difference between training and test data indicating that it is not suffering too much from overfit.

It is the most intensive of the models we tested, however students will not change that much over the course of the year and it is unlikely we will be running such analysis on exceedingly large numbers of students very frequently so this is a very reasonable model for our scale. Should we add features or classify millions of students in real time, we may need to look at a lighter solution, but I think that is a different problem that can be looked when necessary.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: ** Well, we know that there are numerous factors that contribute to whether or not a student needs help with their studies. So, essentially we take those attributes and we plot out the students according to each attribute. Now, this is hard to visualize, but please bear with me. If we consider only the groups of very similar students, and of those only the groups that contain both students who need intervention and those who do not - that is how a Support Vector Machine looks at the world. Next it finds the best way to separate the very similar students who did and did not need intervention accross all of the different attributes. From there, it makes a generalized set of rules for how all the attributes interact and looks at each new student according to those rules to determine whether or not they need intervention.

Each of these rules we can imagine to be a function (or a line on a two dimensional plot) and the shape of that function(line) will be whatever shape maximizes the distance from each similar student in the edge cases (groups of similar students with both passing and not passing students within them). Imagine a group of children congregating in a group, and someone drawing a chalk line between the children such that the line is as far away as possible from each child. Now, the factors that contribute to passing and not passing are unfortunately not easily represented in two dimensions, and it is not easy for us to visualize more than three, but suffice to say these functions can all be represented together in a function of functions, and just as a multitiude of lines can be represented with a surface or plane, the multitude of funcitons in higher orders can be represented with what's called a hyperplane - unfortunately these are often extremely difficult to represent visually.

So, all of these attributes we have collected on past students arrange them in a difficult to visualize higher order space. And, our SVM then finds the most similar students that both contain passing and non-passing students and separates them with a line that maximizes distance between both.

Before there was refrigeration and efficient transportation, the limit of what could be in a bouquet of flowers was what a person could afford, the flowers in one's limited area, and what flowers were in season. It would be impossible to visualize a bouquet of flowers containing flowers from different times, locations, and costs as they would be unattainable simultaneously. However, painters trained in realism would place flowers from different times and geographies in the same postition, in different places, at different times, carrying only the canvas with the unfinished portrait with them and paint a portrait of a bouquet that transcends location, cost, and time, allowing the viewer to look a the most beautiful bouquet without those restraints. With this the artist is creating a hyperplane through the space of geography, availability, and seasonality such that on one side of the hyperplane we have aesthetic in this bouquet and on the other non-aesthetic. I find this to be a fitting metaphor for what our SVM is doing.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [ ]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.metrics import make_scorer
from sklearn.grid_search import GridSearchCV

# TODO: Create the parameters list you wish to tune
parameters = {'gamma':np.logspace(-9, 3, 13), 'C':np.logspace(-2, 10, 13)}

# TODO: Initialize the classifier
clf = svm.SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: ** The model's final F1 score for training and testing were 0.9754 and 0.8481 respectively after training. This is a 0.22% improvement over the untuned model's testing score and a 13.67% improvement over the untuned model's training score. All in all the score did not improve that much because the untuned model already scored quite high. We do now see a larger gap between the F1 score on the testing and training which does raise some concern about overfitting, but the gap is still quite small and accuracy quite high so I am not deeply concerned about it.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.